<a href="https://colab.research.google.com/github/yaming8002/cat_sound_dl/blob/master/pre_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境設定


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import os
soundlist = os.listdir('/content/drive/My Drive/test_sound')
soundlist

['小貓發情了！碎花要變少女了嗎丨麻糬貓奴貓咪日常 001.wav',
 '来听猫叔打呼噜吧, 非常的治愈哦 标清.wav',
 '貓打架-專業版.wav',
 '貓(喵)老大打架_.wav',
 '噓聲.wav',
 '风声_爱给网_aigei_com.wav',
 'Cat asking for help.wav',
 'Cat Purring Sound Effect!.wav',
 '小猫咪的叫声听不厌啊，屡听不爽.wav',
 '藍貓-發情K.wav',
 'box',
 'box1',
 'test',
 'test2']

In [6]:
pip install pydub librosa

# 音檔靜音切割方法

In [ ]:
from pydub.silence import split_on_silence
import os
import librosa
import wave
import numpy as np
import pylab as plt
from pydub import AudioSegment
class mute_cut:
  # 依照聲音檔中的靜音來做切割

  def __init__(self, path= "" ):
      self.isfile = False
      self.have_wav = False
      self.path = path
      if os.path.isfile(path ):
        # 如果是檔案路徑
        # 將檔案讀近來來做切割
        self.isfile = True
        if ".wav" in path:
          self.data = self.sound_split( path )
          self.onename = (path.split("/")[-1]).split(".")[0]
          self.have_wav = True
        else:
          print("此路徑的檔案不是wav檔")
      elif os.path.isdir(path ):
        # 如果是資料夾路徑
        # 紀錄底下所有資料名稱
        self.filename=[ x for x in os.listdir(path) if ".wav" in x]
        if len(self.filename) == 0:
          print("此資料夾沒有wav檔")
        else:
          self.have_wav = True
      

  def sound_split(self,filepath):
      sound = AudioSegment.from_wav(filepath)
      dBFS = sound.dBFS # 指定最大聲音
      # print(dBFS)
      chunks = split_on_silence(sound, 
          min_silence_len = 1000,
          silence_thresh = dBFS-16)
      return chunks

  def output_cutting(self,path_to=""):
      if not self.have_wav :
        print("沒有正確的資料進行分割")
        return None
      if path_to == "":
        path_to = "."
      elif path_to != "" and not os.path.exists(path_to): 
        # 判斷目錄是否存在，不存在就建立檔案
        os.makedirs(path_to)
        
      if self.isfile :
        # 檔案路徑處理
        for i in range(len(self.data)):
          self.data[i].export( f"{path_to}/{self.onename}_{i}.wav", format="wav")

      else:
        # 資夾路徑的處理
        for i in range(len(self.filename)):
          data = self.sound_split( self.path+"/"+self.filename[i])
          name = (self.filename[i].split("/")[-1]).split(".")[0]
          for j in range(len(data)):
            data[j].export( f"{path_to}/{name}_{j}.wav", format="wav")



> 測試


In [ ]:
temp  = mute_cut("/content/drive/My Drive/test_sound/貓(喵)老大打架_.wav")
temp.output_cutting("/content/drive/My Drive/test_sound/test")

# 音檔固定長度切割

In [7]:
from pydub.silence import split_on_silence
import os
import librosa
import wave
import numpy as np
import pylab as plt
from pydub import AudioSegment

def create_data(path_from,path_to,item_len = 2000):
    if path_from.endswith('.wav'):# 輸入的資料來源如果是.wav
        files = [path_from]
    else: # 輸入的資料來源如果是資料夾
        files = os.listdir(path_from)
        files = [f"{path_from}/{ f}" for f in files if f.endswith('.wav')]
    if  not os.path.exists(path_to): # 判斷目錄是否存在，不存在就建立檔案
        os.mkdir(path_to)
    
    for file in files:
        file_name = "".join((file.split('/').pop()).split("_")[:3])
        file_name = file_name.split('.')[0]
        audio = AudioSegment.from_file(file)
        audio_len = len(audio)

        steps = audio_len // item_len+1
        for step in range(steps):
            item_audio = audio[step * item_len:( (step + 1) * item_len if (step + 1) * item_len < audio_len else audio_len )]
            save_audio = item_audio + AudioSegment.silent(item_len - len(item_audio))
            new_path = f"{path_to}/{file_name}_{step}.wav"
            save_audio.export( new_path)
    
    

>測試

In [ ]:
path_from = "/content/drive/My Drive/test_sound/貓(喵)老大打架_.wav"
path_to = "/content/drive/My Drive/test_sound/test2"
create_data(path_from , path_to)

# 聲音資料整合成npy檔

練習資料&驗證資料

In [3]:
import pandas as pd

In [14]:
from pydub import AudioSegment
import os
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
import random
import librosa


x_train_o=[]
y_train_o=[]
x_ver_o=[]
y_ver_o=[]
path = "/content/drive/My Drive/all_data_used"
list_sigle =['none', 'purring', 'cat_talk_cats', 'cat_call_persion', 'caveat']
used_data=[] # 存放用過的檔名

all_data_folder_name = [f"{item}" for item in os.listdir(path) if item in list_sigle  ]
for folder_name in all_data_folder_name:
  # 一個資料夾(類別)隨機選400個出來
  file_name = os.listdir(f"{path}/{folder_name}")
  data_temp = [f"{path}/{folder_name}/{item}" for item in file_name]
  print(folder_name)
  all_data_name =random.sample(data_temp, 400)
  # 記錄用過的檔案
  choise_name = [x.split("/")[-1] for x in all_data_name ]
  used_data += choise_name

  count = 0
  for item in all_data_name: 
    wave, sr = librosa.load(item, mono=True, sr=None)
    zero_np = np.zeros(5)
    zero_np[list_sigle.index(folder_name)]=1
    # 抽取一部分當作驗證資料 9:1
    if count % 10==0:
        x_ver_o.append([wave])
        y_ver_o.append(zero_np)
    else:
        x_train_o.append([wave])
        y_train_o.append(zero_np)
    count +=1

x_train_o = np.array(x_train_o)
y_train_o = np.array(y_train_o)
x_ver_o = np.array(x_ver_o)
y_ver_o = np.array(y_ver_o)

np.save(f'{path}/x_train_o.npy',x_train_o)
np.save(f'{path}/y_train_o.npy',y_train_o)
np.save(f'{path}/x_verification_o.npy',x_ver_o)
np.save(f'{path}/y_verification_o.npy',y_ver_o)

df = pd.DataFrame({"used_name": used_data})
df.to_csv(f'{path}/used_data.csv')


none
caveat
cat_talk_cats
cat_call_persion
purring


>測試資料

In [15]:
x_test_o=[]
y_test_o=[]

all_data_folder_name = [f"{item}" for item in os.listdir(path) if item in list_sigle  ]
print()
for folder_name in all_data_folder_name:
  # 一個資料夾(類別)隨機選400個出來
  file_name = os.listdir(f"{path}/{folder_name}")
  data_temp = [f"{path}/{folder_name}/{item}" for item in file_name if item not in used_data ]
  print(folder_name)
  print(len(data_temp ))
  all_data_name =random.sample(data_temp, 20)
  # 記錄用過的檔案
  used_data += all_data_name

  for item in all_data_name: 
    wave, sr = librosa.load(item, mono=True, sr=None)
    zero_np = np.zeros(5)
    zero_np[list_sigle.index(folder_name)]=1

    x_test_o.append([wave])
    y_test_o.append(zero_np)

x_test_o = np.array(x_test_o)
y_test_o = np.array(y_test_o)
np.save(f'{path}/x_test_o.npy',x_test_o)
np.save(f'{path}/y_test_o.npy',y_test_o)


none
641
caveat
194
cat_talk_cats
132
cat_call_persion
21
purring
161


後續要混的雜訊

In [17]:
from pydub import AudioSegment
import os
import numpy as np
import scipy.io.wavfile as wav
import random
import librosa
import gc
to_niose_data =[]
file_name = os.listdir(f"{path}/none")
# 取出其他雜音，去掉使用過的
data_temp = [f"{path}/none/{item}" for item in os.listdir(f"{path}/none") if item not in used_data ]

for item in data_temp:
  wave, sr = librosa.load(item, mono=True, sr=None)
  to_niose_data.append([wave])
to_niose_data = np.array(to_niose_data)
np.save(f'{path}/to_niose_data.npy',to_niose_data)